<a href="https://colab.research.google.com/github/vgaquino/capstone-team-29/blob/main/kosmos_2_patch14_224.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
import nltk
from nltk.corpus import stopwords
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Demo with caption generation and phrase grounding https://huggingface.co/microsoft/kosmos-2-patch14-224

model = AutoModelForVision2Seq.from_pretrained("microsoft/kosmos-2-patch14-224")
processor = AutoProcessor.from_pretrained("microsoft/kosmos-2-patch14-224")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.45k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.66G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/32.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
start_time = time.time()

In [ ]:
prompt = "<grounding>An image of"

url = "https://huggingface.co/microsoft/kosmos-2-patch14-224/resolve/main/snowman.png"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
image.save("new_image.jpg")
image = Image.open("new_image.jpg")


In [ ]:
inputs = processor(text=prompt, images=image, return_tensors="pt") #extracted from project source repository: https://huggingface.co/microsoft/kosmos-2-patch14-224

generated_ids = model.generate(
    pixel_values=inputs["pixel_values"],
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    image_embeds=None,
    image_embeds_position_mask=inputs["image_embeds_position_mask"],
    use_cache=True,
    max_new_tokens=128,
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]


In [ ]:
processed_text = processor.post_process_generation(generated_text, cleanup_and_extract=False)

In [ ]:
processed_text

'<grounding> An image of<phrase> a snowman</phrase><object><patch_index_0044><patch_index_0863></object> warming himself by<phrase> a fire</phrase><object><patch_index_0005><patch_index_0911></object>.'

In [ ]:
processed_text, entities = processor.post_process_generation(generated_text)

In [ ]:
processed_text

'An image of a snowman warming himself by a fire.'

In [ ]:
entities

[('a snowman', (12, 21), [(0.390625, 0.046875, 0.984375, 0.828125)]),
 ('a fire', (41, 47), [(0.171875, 0.015625, 0.484375, 0.890625)])]

In [ ]:
processed_text = processor.post_process_generation(generated_text, cleanup_and_extract=False)

print(processed_text)
# `<grounding> An image of<phrase> a snowman</phrase><object><patch_index_0044><patch_index_0863></object> warming himself by<phrase> a fire</phrase><object><patch_index_0005><patch_index_0911></object>.`

# By default, the generated  text is cleanup and the entities are extracted.
processed_text, entities = processor.post_process_generation(generated_text)

print(processed_text)
# `An image of a snowman warming himself by a fire.`

print(entities)


<grounding> An image of<phrase> a snowman</phrase><object><patch_index_0044><patch_index_0863></object> warming himself by<phrase> a fire</phrase><object><patch_index_0005><patch_index_0911></object>.
An image of a snowman warming himself by a fire.
[('a snowman', (12, 21), [(0.390625, 0.046875, 0.984375, 0.828125)]), ('a fire', (41, 47), [(0.171875, 0.015625, 0.484375, 0.890625)])]


In [ ]:
entities_list = [i[0] for i in entities]
print(entities_list)

['a snowman', 'a fire']


In [ ]:
entity_list = []
[entity_list.extend(idx.split(" ")) for idx in entities_list]
entity_list

['a', 'snowman', 'a', 'fire']

In [ ]:
import nltk
from nltk.corpus import stopwords

# Download stopwords from NLTK
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

filtered_sentence = [w for w in entity_list if not w.lower() in stop_words]
filtered_sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['snowman', 'fire']

In [ ]:
end_time = time.time()

In [ ]:
total_time = end_time - start_time
print(f"Total time taken to process 1 image: {total_time} seconds")

Total time taken to process 1 image: 34.89652729034424 seconds


In [ ]:
## Original code that shows LookupError
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))

# filtered_sentence = [w for w in entity_list if not w.lower() in stop_words]
# filtered_sentence

Estimate the time and computational power needed to process five images.

In [ ]:
prompt = "<grounding>An image of"

# Random images for demo
image_urls = ["https://huggingface.co/microsoft/kosmos-2-patch14-224/resolve/main/snowman.png",
              "https://www.thestrapsaver.com/wp-content/uploads/customerservice-1.jpg",
              "https://fastly.picsum.photos/id/9/5000/3269.jpg?hmac=cZKbaLeduq7rNB8X-bigYO8bvPIWtT-mh8GRXtU3vPc",
              "https://fastly.picsum.photos/id/64/4326/2884.jpg?hmac=9_SzX666YRpR_fOyYStXpfSiJ_edO3ghlSRnH2w09Kg",
              "https://fastly.picsum.photos/id/75/1999/2998.jpg?hmac=0agRZd8c5CRiFvADOWJqfTv6lqYBty3Kw-9LEtLp_98"]

In [ ]:
start_time = time.time()

In [ ]:
%%time

image_entities_dict = {}

for url in image_urls:
    # Download image
    image = Image.open(requests.get(url, stream=True).raw)

    # Save the images
    image_filename = url.split("/")[-1].split("?")[0]
    image.save(image_filename)

    # Load the image for processing
    image = Image.open(image_filename)

    # Process image
    inputs = processor(text=prompt, images=image, return_tensors="pt")

    generated_ids = model.generate(
    pixel_values=inputs["pixel_values"],
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    image_embeds=None,
    image_embeds_position_mask=inputs["image_embeds_position_mask"],
    use_cache=True,
    max_new_tokens=128,
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    processed_text = processor.post_process_generation(generated_text, cleanup_and_extract=False)
    processed_text, entities = processor.post_process_generation(generated_text)

    entities_list = [i[0] for i in entities]

    entity_list = []
    [entity_list.extend(idx.split(" ")) for idx in entities_list]

    stop_words = set(stopwords.words('english'))
    filtered_sentence = [w for w in entity_list if not w.lower() in stop_words]

    image_entities_dict[image_filename] = filtered_sentence

print(image_entities_dict)

end_time = time.time()

{'snowman.png': ['snowman', 'campfire'], 'customerservice-1.jpg': ['hanger', 'words', '"We', 'love', 'customers"'], '3269.jpg': ['laptop', 'cell', 'phone', 'glass', 'wooden', 'table'], '2884.jpg': ['beautiful', 'girl', 'sunglasses'], '2998.jpg': ['grapes', 'vine']}
CPU times: user 1min 59s, sys: 1.5 s, total: 2min
Wall time: 33.2 s


In [ ]:
list_length = len(image_urls)

total_time = end_time - start_time
hours, remainder = divmod(total_time, 3600)
minutes, seconds = divmod(remainder, 60)
print(f"Total time taken to process {list_length} images: {int(hours)} hours {int(minutes)} minutes {seconds} seconds")

Total time taken to process 5 images: 0 hours 0 minutes 33.493802070617676 seconds


In [ ]:
# !unzip -q -o '/content/drive/MyDrive/Colab Notebooks/train.zip' -d images
from google.colab import drive
drive.mount('/content/drive')
!unzip -q -o '/content/drive/My Drive/Colab Notebooks/val.zip' -d images
!unzip -q -o '/content/drive/My Drive/Colab Notebooks/test.zip' -d images


Mounted at /content/drive
